In [3]:
import pandas as pd

# Load the three datasets
seen_file_path = 'papers_seen.csv'
downloaded_file_path = 'papers_downloaded.csv'
kept_file_path = 'papers_kept.csv'

papers_seen = pd.read_csv(seen_file_path)
papers_downloaded = pd.read_csv(downloaded_file_path)
papers_kept = pd.read_csv(kept_file_path)

In [7]:
# Find rows in papers_downloaded that are not in papers_seen
discrepancies_downloaded_not_in_seen = papers_downloaded.merge(papers_seen, how='left', indicator=True)
discrepancies_downloaded_not_in_seen = discrepancies_downloaded_not_in_seen[discrepancies_downloaded_not_in_seen['_merge'] == 'left_only']

# Display the discrepancies
discrepancies_downloaded_not_in_seen.drop(columns=['_merge'], inplace=True)
#import ace_tools as tools; tools.display_dataframe_to_user(name="Discrepancies in papers_downloaded not found in papers_seen", dataframe=discrepancies_downloaded_not_in_seen)


In [11]:
# First, manually add the missing rows from papers_downloaded to papers_seen in the working representation
papers_seen_updated = pd.concat([papers_seen, discrepancies_downloaded_not_in_seen], ignore_index=True).drop_duplicates()

# Now, create the two extra columns:
# 1. 'In Downloaded' which is 1 if the row exists in papers_downloaded, 0 otherwise
# 2. 'In Kept' which is 1 if the row exists in papers_kept, 0 otherwise

# First, we'll join papers_seen with papers_downloaded and papers_kept to correctly add the flags
papers_dataset = papers_seen_updated.copy()

# Adding the 'In Downloaded' column
papers_dataset = papers_dataset.merge(
    papers_downloaded.assign(In_Downloaded=1), 
    on=papers_downloaded.columns.tolist(), 
    how='left'
).fillna({'In_Downloaded': 0})

# Adding the 'In Kept' column
papers_dataset = papers_dataset.merge(
    papers_kept.assign(In_Kept=1), 
    on=papers_kept.columns.tolist(), 
    how='left'
).fillna({'In_Kept': 0})

# Save the updated dataset as 'papers_dataset.csv'
papers_dataset_path_corrected = 'papers_dataset.csv'
papers_dataset.to_csv(papers_dataset_path, index=False)

papers_dataset_path


'papers_dataset_corrected.csv'